In [1]:
import requests

In [2]:
standings_url = "https://shorttrack.sportresult.com/?evt=11213100000088&__VIEWSTATEGENERATOR=CA0B0334"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
links_a = soup.find("div", {"id": "contentrightsmall"}).find_all('a')

In [7]:
links = ["https://shorttrack.sportresult.com/" + l.get("href") for l in links_a]

In [8]:
names = [n.contents[0] for n in links_a]

各比赛url，进入各比赛

In [9]:
link = links[9]
name = names[9]

In [10]:
link

'https://shorttrack.sportresult.com/Results.aspx?evt=11213100000088&gen=w&ref=64868&view=rou'

In [11]:
name

'Women - 500m - Finals'

In [12]:
sex, project, level = name.split(" - ")

In [13]:
sex, project, level

('Women', '500m', 'Finals')

In [14]:
result_data = requests.get(link)

In [15]:
import pandas as pd

Final A B 的表格

In [16]:
tables = pd.read_html(result_data.text)[2:] # 这里希望通过match 匹配正确的表格，并且想要获取 Final A　＆　Final　B　的名字

爬取 两个spllits 的url

In [17]:
split_data = BeautifulSoup(result_data.text).find_all("div", {"class": "tabletitle"})

In [18]:
split_links = [l.find("a").get("href") for l in split_data]

In [19]:
split_links

['http://shorttrack.sportresult.com/Results.aspx?evt=11213100000088&gen=w&ref=64868&view=rou&sub=spl&rac=1',
 'http://shorttrack.sportresult.com/Results.aspx?evt=11213100000088&gen=w&ref=64868&view=rou&sub=spl&rac=2']

爬取 表格的名称 eg Final A

In [20]:
import re
table_names = [re.sub('[\r\n\t ]','',table.find('b').text) for table in split_data]

['FinalA', 'FinalB']

爬取里面的表格

In [21]:
split_link = split_links[0]

In [22]:
split_data = requests.get(split_link)

In [23]:
split_tables = pd.read_html(split_data.text)
split_table = pd.read_html(split_data.text)[1]

2

In [24]:
import numpy as np

In [25]:
split_table_T = split_table.set_index("Unnamed: 0").T

In [26]:
split_table_T

Unnamed: 0,1,2,3,4,5
Arianna FONTANA,[2] 7.10 (7.10),[2] 16.19 (9.09),[2] 24.87 (8.68),[1] 33.62 (8.75),[1] 42.488 (8.86)
Suzanne SCHULTING,[1] 6.96 (6.96),[1] 16.07 (9.11),[1] 24.79 (8.72),[2] 33.69 (8.90),[2] 42.559 (8.86)
BOUTIN Kim,[3] 7.23 (7.23),[3] 16.30 (9.07),[3] 24.99 (8.69),[3] 33.79 (8.80),[3] 42.724 (8.93)
ZHANG Yuting,[4] 7.44 (7.44),[4] 16.50 (9.06),[4] 25.10 (8.60),[4] 33.86 (8.76),[4] 42.803 (8.94)
DESMET Hanne,[5] 7.66 (7.66),[5] 16.78 (9.12),[5] 25.35 (8.57),[5] 34.05 (8.70),[5] 42.941 (8.89)


In [27]:
import re


In [28]:
x, y = split_table_T.shape
lap_time = split_table_T
for i in range(x):
    for j in range(y):
        m = re.match(r'\[(\d)\]\s+(\d+\.\d+)\s+\((\d+\.\d+)\)', split_table_T.iloc[i, j])
        lap_time.iloc[i, j] = m.group(3)

In [29]:
lap_time

Unnamed: 0,1,2,3,4,5
Arianna FONTANA,7.10,9.09,8.68,8.75,8.86
Suzanne SCHULTING,6.96,9.11,8.72,8.90,8.86
BOUTIN Kim,7.23,9.07,8.69,8.80,8.93
ZHANG Yuting,7.44,9.06,8.60,8.76,8.94
DESMET Hanne,7.66,9.12,8.57,8.70,8.89


In [30]:
lap_time = lap_time.rename_axis('index').reset_index().rename({"index": "Name"}, axis=1)

In [31]:
lap_time

Unnamed: 0,Name,1,2,3,4,5
0,Arianna FONTANA,7.10,9.09,8.68,8.75,8.86
1,Suzanne SCHULTING,6.96,9.11,8.72,8.90,8.86
2,BOUTIN Kim,7.23,9.07,8.69,8.80,8.93
3,ZHANG Yuting,7.44,9.06,8.60,8.76,8.94
4,DESMET Hanne,7.66,9.12,8.57,8.70,8.89


In [32]:
pd.merge(tables[0], lap_time, how="inner")

,Place,Start Pos.,Warn.,#,Name,ISU Member,Results,Unnamed: 7,Qual.,Points,1,2,3,4,5
0,1,1,NaN,3,Arianna FONTANA,ITA,42.488,NaN,NaN,NaN,7.10,9.09,8.68,8.75,8.86
1,2,2,NaN,1,Suzanne SCHULTING,NED,42.559,NaN,NaN,NaN,6.96,9.11,8.72,8.90,8.86
2,3,3,NaN,35,BOUTIN Kim,CAN,42.724,NaN,NaN,NaN,7.23,9.07,8.69,8.80,8.93
3,4,4,NaN,45,ZHANG Yuting,CHN,42.803,NaN,NaN,NaN,7.44,9.06,8.60,8.76,8.94
4,5,5,NaN,5,DESMET Hanne,BEL,42.941,NaN,NaN,NaN,7.66,9.12,8.57,8.70,8.89
